In [1]:
import pandas as pd
import glob, os
from pandas.io.json import json_normalize as jn
import itertools
import json
import requests
import bz2
# data schema: https://historicdata.betfair.com/Betfair-Historical-Data-Feed-Specification.pdf"

In [2]:
# Set view parameters
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
path = '/Users/henry/code/freelance/upwork/04.03_betfair_download/data'

In [4]:
# ALTERNATIVE
# files_lists = [glob.glob(os.path.join(path, dir + '/*.bz2')) for dir in dirs]
# files = list(itertools.chain(*files_lists))

In [5]:
# General - set all .bz2 files in data folder as 'files'
files = glob.glob(os.path.join(path + '/**/*.bz2'), recursive=True)

In [8]:
# Specific - set all .bz2 files in data/201811M as 'test_files'
test_files = glob.glob(os.path.join(path + '/2018_11/*.bz2'))

In [23]:
# TEMPORARY - FOR TESTING ONLY
test_file = test_files[6]
test_file2 = '/Users/henry/code/freelance/upwork/04.03_betfair_download/data/2018_11/1.149382818.bz2'

#### Uncompress bz2 data into bytes with BZ2File.read()

In [24]:
test_file = 'data/2016_07/1.125411349.bz2'

In [25]:
uncompressedData = bz2.BZ2File(test_file).read()
uncompressedData

OSError: Invalid data stream

In [25]:
type(uncompressedData)

bytes

#### Turn bytes into string with .decode()

In [26]:
data = uncompressedData.decode('utf8')

#### Read bz2 into df with pd.read_json

In [27]:
df = pd.read_json(test_file, lines=True, compression='bz2')

In [28]:
df.shape

(123, 4)

In [29]:
df.columns

Index(['clk', 'mc', 'op', 'pt'], dtype='object')

In [30]:
# Change column names
df.columns = ['sequence_token', 'market_change', 'operation_type', 'published_time']

In [32]:
# Expand market_change column into df
market_change = jn(df['market_change'][0])

for x in range(1, df.shape[0]):
    flat = jn(df['market_change'][x])
    market_change = market_change.append(flat, sort=False)

In [33]:
market_change.shape

(123, 31)

In [35]:
market_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 0 to 0
Data columns (total 31 columns):
id                                        123 non-null object
marketDefinition.betDelay                 5 non-null float64
marketDefinition.bettingType              5 non-null object
marketDefinition.bspMarket                5 non-null object
marketDefinition.bspReconciled            5 non-null object
marketDefinition.complete                 5 non-null object
marketDefinition.countryCode              5 non-null object
marketDefinition.crossMatching            5 non-null object
marketDefinition.discountAllowed          5 non-null object
marketDefinition.eventId                  5 non-null object
marketDefinition.eventName                5 non-null object
marketDefinition.eventTypeId              5 non-null object
marketDefinition.inPlay                   5 non-null object
marketDefinition.marketBaseRate           5 non-null float64
marketDefinition.marketTime               5 non-null 

In [36]:
market_change.dropna(axis=0, subset=['marketDefinition.bettingType']).reset_index(drop=True)

,id,marketDefinition.betDelay,marketDefinition.bettingType,marketDefinition.bspMarket,marketDefinition.bspReconciled,marketDefinition.complete,marketDefinition.countryCode,marketDefinition.crossMatching,marketDefinition.discountAllowed,marketDefinition.eventId,marketDefinition.eventName,marketDefinition.eventTypeId,marketDefinition.inPlay,marketDefinition.marketBaseRate,marketDefinition.marketTime,marketDefinition.marketType,marketDefinition.name,marketDefinition.numberOfActiveRunners,marketDefinition.numberOfWinners,marketDefinition.openDate,marketDefinition.persistenceEnabled,marketDefinition.regulators,marketDefinition.runners,marketDefinition.runnersVoidable,marketDefinition.status,marketDefinition.suspendTime,marketDefinition.timezone,marketDefinition.turnInPlayEnabled,marketDefinition.version,rc,marketDefinition.settledTime
0,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN
1,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN
2,1.151140400,5.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,True,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.528410e+09,NaN,NaN
3,1.151140400,5.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,True,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,SUSPENDED,2018-11-24T15:00:00.000Z,Europe/London,True,2.528994e+09,"[{'ltp': 14.5, 'id': 58805}, {'ltp': 1.09, 'id...",NaN
4,1.151140400,5.0,ODDS,False,False,True,GB,False,True,29007101,AFC Fylde v Boreham Wood,1,True,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,0.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'WINNER', 'sortPriority': 1, 'id':...",False,CLOSED,2018-11-24T15:00:00.000Z,Europe/London,True,2.529009e+09,NaN,2018-11-24T16:54:12.000Z


In [37]:
# Expand runners - match odds here?

In [40]:
# Add expanded market change column to previous df
df = df.copy().join(market_change).drop('market_change', axis=1).dropna(axis=0, subset=['marketDefinition.bettingType']).reset_index(drop=True)

-------------------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------------------

### Long way to achieve same as pd.read_json()

In [56]:
# Create list of rows as strings in .bz2 file
with bz2.open(test_file, 'rt') as f:
    rowlist = [line.rstrip('') for line in f]

f.close()

# Turn list of rows as strings into list of rows as dictionaries
json_list = [json.loads(x) for x in testlist]

# Turn list of rows as dictionaries into list of flattened dfs
df_list = []

for j in json_list:
    temp = jn(j)
    temp2 = temp.join(jn(temp['mc'][0]))
    df_list.append(temp2)

# Turn list of flattened dfs into single df
json_df = pd.concat(df_list, sort=False).reset_index(drop=True)

In [58]:
# Inspect df
json_df.head()

,clk,mc,op,pt,id,marketDefinition.betDelay,marketDefinition.bettingType,marketDefinition.bspMarket,marketDefinition.bspReconciled,marketDefinition.complete,marketDefinition.countryCode,marketDefinition.crossMatching,marketDefinition.discountAllowed,marketDefinition.eventId,marketDefinition.eventName,marketDefinition.eventTypeId,marketDefinition.inPlay,marketDefinition.marketBaseRate,marketDefinition.marketTime,marketDefinition.marketType,marketDefinition.name,marketDefinition.numberOfActiveRunners,marketDefinition.numberOfWinners,marketDefinition.openDate,marketDefinition.persistenceEnabled,marketDefinition.regulators,marketDefinition.runners,marketDefinition.runnersVoidable,marketDefinition.status,marketDefinition.suspendTime,marketDefinition.timezone,marketDefinition.turnInPlayEnabled,marketDefinition.version,rc,marketDefinition.settledTime
0,7966461884,"[{'id': '1.151140400', 'marketDefinition': {'b...",mcm,1542102125612,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN
1,8036141622,"[{'id': '1.151140400', 'rc': [{'ltp': 2.0, 'id...",mcm,1542934975186,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 2.0, 'id': 4317866}]",NaN
2,8038907063,"[{'id': '1.151140400', 'rc': [{'ltp': 4.4, 'id...",mcm,1542969181853,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 4.4, 'id': 256120}, {'ltp': 3.8, 'id'...",NaN
3,8042717121,"[{'id': '1.151140400', 'rc': [{'ltp': 4.5, 'id...",mcm,1542998124690,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 4.5, 'id': 256120}, {'ltp': 3.6, 'id'...",NaN
4,8043359071,"[{'id': '1.151140400', 'marketDefinition': {'b...",mcm,1543001878564,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN


In [89]:
# Print runners ignoring nans
for x in json_df['marketDefinition.runners']:
    if type(x) == list:
        print(x)

[{'status': 'ACTIVE', 'sortPriority': 1, 'id': 4317866, 'name': 'AFC Fylde'}, {'status': 'ACTIVE', 'sortPriority': 2, 'id': 256120, 'name': 'Boreham Wood'}, {'status': 'ACTIVE', 'sortPriority': 3, 'id': 58805, 'name': 'The Draw'}]
[{'status': 'ACTIVE', 'sortPriority': 1, 'id': 4317866, 'name': 'AFC Fylde'}, {'status': 'ACTIVE', 'sortPriority': 2, 'id': 256120, 'name': 'Boreham Wood'}, {'status': 'ACTIVE', 'sortPriority': 3, 'id': 58805, 'name': 'The Draw'}]
[{'status': 'ACTIVE', 'sortPriority': 1, 'id': 4317866, 'name': 'AFC Fylde'}, {'status': 'ACTIVE', 'sortPriority': 2, 'id': 256120, 'name': 'Boreham Wood'}, {'status': 'ACTIVE', 'sortPriority': 3, 'id': 58805, 'name': 'The Draw'}]
[{'status': 'ACTIVE', 'sortPriority': 1, 'id': 4317866, 'name': 'AFC Fylde'}, {'status': 'ACTIVE', 'sortPriority': 2, 'id': 256120, 'name': 'Boreham Wood'}, {'status': 'ACTIVE', 'sortPriority': 3, 'id': 58805, 'name': 'The Draw'}]
[{'status': 'WINNER', 'sortPriority': 1, 'id': 4317866, 'name': 'AFC Fylde'}

In [90]:
for x in json_df['marketDefinition.runners']:
    top_level = top_level.join(jn(x), rsuffix='_x')

TypeError: 'float' object is not iterable

#### Some other attempt

In [77]:
crazy = pd.concat(df_list, sort=False).reset_index(drop=True)

In [78]:
crazy.loc[:5]

,clk,mc,op,pt,id,marketDefinition.betDelay,marketDefinition.bettingType,marketDefinition.bspMarket,marketDefinition.bspReconciled,marketDefinition.complete,marketDefinition.countryCode,marketDefinition.crossMatching,marketDefinition.discountAllowed,marketDefinition.eventId,marketDefinition.eventName,marketDefinition.eventTypeId,marketDefinition.inPlay,marketDefinition.marketBaseRate,marketDefinition.marketTime,marketDefinition.marketType,marketDefinition.name,marketDefinition.numberOfActiveRunners,marketDefinition.numberOfWinners,marketDefinition.openDate,marketDefinition.persistenceEnabled,marketDefinition.regulators,marketDefinition.runners,marketDefinition.runnersVoidable,marketDefinition.status,marketDefinition.suspendTime,marketDefinition.timezone,marketDefinition.turnInPlayEnabled,marketDefinition.version,rc,marketDefinition.settledTime
0,7966461884,"[{'id': '1.151140400', 'marketDefinition': {'b...",mcm,1542102125612,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN
1,8036141622,"[{'id': '1.151140400', 'rc': [{'ltp': 2.0, 'id...",mcm,1542934975186,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 2.0, 'id': 4317866}]",NaN
2,8038907063,"[{'id': '1.151140400', 'rc': [{'ltp': 4.4, 'id...",mcm,1542969181853,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 4.4, 'id': 256120}, {'ltp': 3.8, 'id'...",NaN
3,8042717121,"[{'id': '1.151140400', 'rc': [{'ltp': 4.5, 'id...",mcm,1542998124690,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 4.5, 'id': 256120}, {'ltp': 3.6, 'id'...",NaN
4,8043359071,"[{'id': '1.151140400', 'marketDefinition': {'b...",mcm,1543001878564,1.151140400,0.0,ODDS,False,False,True,GB,True,True,29007101,AFC Fylde v Boreham Wood,1,False,5.0,2018-11-24T15:00:00.000Z,MATCH_ODDS,Match Odds,3.0,1.0,2018-11-24T15:00:00.000Z,True,[MR_INT],"[{'status': 'ACTIVE', 'sortPriority': 1, 'id':...",False,OPEN,2018-11-24T15:00:00.000Z,Europe/London,True,2.512030e+09,NaN,NaN
5,8044858666,"[{'id': '1.151140400', 'rc': [{'ltp': 3.8, 'id...",mcm,1543012051066,1.151140400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'ltp': 3.8, 'id': 58805}, {'ltp': 1.99, 'id'...",NaN
